In [1]:
#import libraries
import geopandas as gpd
import numpy as np 
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.figure_factory as ff
import plotly.colors

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "simple_white"

In [2]:
# read file
df = pd.read_csv('./data/us-counties.csv')

In [3]:
#df.head()

In [4]:
# remove nan and null
df.dropna(inplace = True)

In [5]:
# convert to states
df_state = df.groupby('state').sum()
df_state = pd.DataFrame(df_state)
df_state = df_state.reset_index()

In [6]:
# decode states 
# using https://stackoverflow.com/questions/48979352/choropleth-map-in-python-using-plotly-without-state-codes
state_codes = {
    'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 
    'Nevada': 'NV', 'Maine': 'ME', 'Northern Mariana Islands' : 'MP', 'Puerto Rico': 'PR', 'Virgin Islands' : 'VI'}

In [7]:
df_state['state_code'] = df_state['state'].apply(lambda x : state_codes[x])

In [8]:
df_state.head()
#print(px.colors.sequential.purples)

,state,fips,cases,deaths,state_code
0,Alabama,23793216.0,54182485,880309.0,AL
1,Alaska,17102510.0,5465696,25348.0,AK
2,Arizona,20531132.0,82564902,1665484.0,AZ
3,Arkansas,125087797.0,32012787,516708.0,AR
4,California,119297722.0,330687073,4956104.0,CA


In [9]:
data = dict(type='choropleth', 
            colorscale='peach', 
            locations = df_state['state_code'], 
            locationmode = 'USA-states', 
            z = df_state['cases'], 
            text = df_state['state'], 
            colorbar={'title':'Scale'},  
            marker = dict(line=dict(width=0))) 
layout = dict(title = 'COVID 19 Cases by States', 
              geo = dict(scope='usa', 
                         subunitcolor="Black"))
state_map = go.Figure(data = [data], layout = layout)
iplot(state_map)

In [10]:
top_10 = df_state.sort_values(by = ['cases'], ascending=False)[0:10]
#top_10.head(10)

In [11]:
min_10 = df_state.sort_values(by = ['cases'], ascending=False)[-10:]
#min_10.head(10)

In [12]:
top_10_d = df_state.sort_values(by = ['deaths'], ascending=False)[0:10]
#top_10_d.head(10)

In [13]:
min_10_d = df_state.sort_values(by = ['deaths'], ascending=False)[-10:]
#min_10_d.head(10)

In [14]:
fig = make_subplots(rows = 1, cols = 2, 
    specs = [[{"type": "bar"}, {"type": "bar"}]],
    subplot_titles = ("Top 10 States with Confirmed Cases","10 States with Minimum Confirmed Cases"))

fig.add_trace(go.Bar(x = top_10['state'], y = top_10['cases'],marker=dict(color='rgb(239, 106, 76)')), 
              row=1, col=1)

fig.add_trace(go.Bar(x = min_10['state'], y = min_10['cases'],marker=dict(color='rgb(250, 203, 166)')),
              row=1, col=2)

fig.update_layout(coloraxis_autocolorscale=False, coloraxis_colorscale = 'peach', title_text="US states by Covid-19 Cases", showlegend=False)

iplot(fig)

In [15]:
data = dict(type='choropleth', 
            colorscale='purples', 
            locations = df_state['state_code'], 
            locationmode = 'USA-states', 
            z = df_state['deaths'], 
            text = df_state['state'], 
            colorbar={'title':'Scale'},  
            marker = dict(line=dict(width=0))) 
layout = dict(title = 'COVID 19 Deaths by States', 
              geo = dict(scope='usa', 
                         subunitcolor="Black"))
state_map = go.Figure(data = [data], layout = layout)
iplot(state_map)

In [16]:
fig = make_subplots(rows = 1, cols = 2, 
    specs = [[{"type": "bar"}, {"type": "bar"}]],
    subplot_titles = ("Top 10 States with Death Cases","10 States with Minimum Death Cases"))

fig.add_trace(go.Bar(x = top_10_d['state'], y = top_10_d['deaths'],marker=dict(color='rgb(63, 12, 123)')),
              row=1, col=1)

fig.add_trace(go.Bar(x = min_10_d['state'], y = min_10_d['deaths'],marker=dict(color='rgb(197, 198, 224)')),
              row=1, col=2)

fig.update_layout(title_text="US states by Covid-19 Death Cases", showlegend=False)

iplot(fig)